# Notebook designed to retireve data from any Snapshot space

In [1]:
import pandas as pd
import requests

# DAO queries and data retrieval 

In [2]:
# Production hub
URL = "https://hub.snapshot.org/graphql"

### "id" of the Snapshot space, i.e. IndieDAO space

In [3]:
DAO = "indiedao.eth"

Save proposals information

In [4]:
def get_all_proposals(dao, date):
  QUERY_VOTES = f"""query Proposals {{
  proposals(
    where: {{space_in: "{dao}", created_gt: {date}}},
    first: 1000,
    orderBy: "created",
    orderDirection: asc
  ) {{
    id
    created
    title
    votes
    type
    state
    author
    scores
    scores_total
    choices
    end
  }}
}}
"""
  return QUERY_VOTES

def set_all_proposals_info(dao):
  date = 0
  list_dic_proposals = []
  while True:
    props = requests.post(URL, json={'query': get_all_proposals(dao, date)}).json()["data"]["proposals"]

    list_dic_proposals += props
    date = max(list(map(lambda x: x["created"], props)))

    if len(props) < 1000:
      break

  df_proposals = pd.DataFrame.from_dict(list_dic_proposals, orient='columns')
  df_proposals['created'] = pd.to_datetime(df_proposals['created'], unit='s')
  df_proposals['end'] = pd.to_datetime(df_proposals['end'], unit='s')
  df_proposals['author'] = df_proposals['author'].str.lower()

  return df_proposals


df_proposals = set_all_proposals_info(DAO)

#Save data in csv format
df_proposals.to_csv("proposals.csv")

Save the voting information

In [5]:
def get_all_votes(dao, date):
  QUERY_VOTES = f"""query Votes {{
    votes(
      where: {{space_in: "{dao}", created_gt: {date} }},
      first: 1000,
      orderBy: "created",
      orderDirection: asc
    ) {{
      id
      created
      voter
      vp
      choice
      proposal {{
          id
          title
      }}
    }}
  }}
  """
  return QUERY_VOTES

def set_votes_info(dao):
  date = 0
  list_dict_results = []
  while True:

    votes = requests.post(URL, json={'query': get_all_votes(dao, date)}).json()["data"]["votes"]
    list_dict_results += votes
    date = max(list(map(lambda x: x["created"], votes)))

    if len(votes) < 1000:
      break

  df_votes = pd.DataFrame.from_dict(list_dict_results, orient='columns')
  df_votes['created'] = pd.to_datetime(df_votes['created'], unit='s')
  df_votes['voter'] = df_votes['voter'].str.lower()
  df_votes["choice"] = df_votes["choice"].apply(lambda val: val[0] if (type(val) == list) else val)
  return df_votes


df_votes = set_votes_info(DAO)

#Save data in csv format
df_votes.to_csv("votes.csv")